# 네이버 지도에서 카페정보 크롤링하기
jupyter notebook을 기준으로 하였습니다.
참고 : https://m.blog.naver.com/tamiblue/221723206818

한 번 처음부터 천천히 진행해보세요!  
크롬창이 어떻게 변하는지 확인하시면 추가적으로 코드를 짜는 데 도움이 될 것 같습니다!  
보통 XPATH로 제가 구성을 해놓았는데, 가끔 안 맞는 경우가 있어서... 셀레니움으로 켜놓는 거 말고 기존 크롬의 개발자도구로 같이 따라가면서 하시는 것도 추천드리는 바입니다!

In [ ]:
#셀레늄 라이브러리 불러오기
from selenium import webdriver

#셀레늄에서 html문서가 생성될 때까지 대기하는 함수 불러오기
from selenium.webdriver.support.ui import WebDriverWait

#셀레늄에서 조건을 걸어줄 수 있는 함수 불러오기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

# 필요한 함수들을 불러줍니다!
import re
import time
from pandas import DataFrame
import pandas as pd
import numpy as np

In [ ]:
# End-To-End 도전!

# 기본 설정하기

# 크롬드라이버 경로 설정
# 경로는 본인의 ch|romedriver로 설정
path = r"C:\Users\김지희\modeling_project\chromedriver.exe"   # 여기 수정!!

#드라이버 객체 생성
browser = webdriver.Chrome(path)

# publicdata_cafe.csv 파일을 불러옵니다! 디렉토리 유의하세요!
df = pd.read_csv('C:\\Users\\김지희\\modeling_project\\publicdata_cafe.csv')   # 여기 수정!

# 네이버 지도 주소 입력
browser.get('https://map.naver.com/v5/')

C:\ProgramData\Anaconda3\envs\Crawling\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # This is added back by InteractiveShellApp.init_path()


아래 부분은 네이버 지도 사이트가 제대로 열린 것을 확인하고 나서 진행해주세요

In [ ]:
# 시간 측정
start = time.time()

# 저희의 모든 정보는 이 final 리스트에 담깁니다!(dictionary가 순서대로 리스트에 들어감)
final = []

for num_iter in range(12200,12220): # 크롤링할 영역에 맞게 수정하세요!
    
    # 앞으로 자주 쓰일 것 같아서 그냥 name이라는 변수로 저장
    name = df.loc[num_iter,'이름']
    
    # 행복한 이스터에그
    if num_iter%100 == 0:
        print('와! {0}개째 Crawling!'.format(num_iter))
        
    # 극단배우다방 일단 저리 치우겠습니다
    if name == '극단배우다방':
        print('극단배우다방 OUT.')
        continue
        
    # 카페의 법정동명을 real_location이라는 변수로 저장 (시군구명으로도 변경가능)
    real_location = df.loc[num_iter,'법정동명']
    
    # 검색창에 접속하기
    search = browser.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    
    # 검색창에 검색어 넣기
    search.clear() # 검색어창 청소
    search.send_keys(name)  # 검색어 입력
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    time.sleep(2)
    
    # 검색결과에 맞춰서 카페 창이 딱 뜬 경우
    try:
        frame = browser.find_element(By.CSS_SELECTOR, "iframe#entryIframe")
        browser.switch_to.frame(frame)
    
    # 카페 창이 바로 안 뜬 경우
    except:
        # searchIframe으로 변경
        frame = browser.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
        browser.switch_to.frame(frame)
                                        
        
        try:
            # 데이터에서 가져온 법정동명과 검색해서 나온 카페 주소 비교
            isnt_location = True
            order = 0
            while isnt_location:
                order += 1
                try: # 리스트에 카페가 2개 이상인 경우
                    location = browser.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[{0}]/div[2]/div'.format(order)).text
                
                except: # 리스트에 카페가 1개인 경우
                    location = browser.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li/div[1]/div[1]/div/span/a/span[1]').text
                
                # 카페주소와 법정동명 일치하면, 바로 그 카페가 우리가 찾는 카페?
                if real_location in location:
                    isnt_location = False
                
                else:
                    # 최대 탐색 개수는 15로 제한
                    if order == 15:
                        browser.switch_to.parent_frame()
                        continue
                 
            # 링크 버튼도 카페 갯수에 따라 XPATH가 다릅니다
            try:
                link_button = browser.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[{0}]/div[2]/a[1]'.format(order))
            except:
                link_button = browser.find_element(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li/div[1]/a')
            
            link_button.click()
            time.sleep(1)
    
            browser.switch_to.parent_frame()
        
            frame = browser.find_element(By.CSS_SELECTOR, 'iframe#entryIframe')
            browser.switch_to.frame(frame)
    
        # 아니면 검색된 링크가 없는 거다
        except:
            print('{0}번 카페 {1}, 이 곳은 망한 것 같습니다.'.format(num_iter, name))
            browser.switch_to.parent_frame()
            continue
        
    
    # 이 아래부분부터 해당카페가 존재하는 경우에만 돌아가게 됩니다!
    
    
    # 새로운 창을 열기전에 가장 가까운 역과 그때의 거리 정보를 가져와봅시다!
    try:
        station = browser.find_element(by=By.CSS_SELECTOR, value = '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin.vKA6F > div > ul > li.SF_Mq.GFtzE').text
        all_address = station.split('\n')
        
        if len(all_address) == 3: # 역 정보까지 있는 경우
            
            all_station = all_address[2].split(' ')
            
            station = all_station[0]
            
            road_address = all_address[1]
            
            station_distance = all_station[-1][4:]
        
        else: # 역정보가 없는 경우
            station = None
            station_distance = None
            road_address = None
            
    except: # 주소칸이 없는경우
        station = None
        station_distance = None
        road_address = None
        pass

    
    # '리뷰' 버튼 클릭하기!
    # 리뷰가 몇 번째에 있을지가 불확실해서
    # 그 부분을 고려하는 내용입니다
    
    # '리뷰' 버튼이 없으면 아예 넘어가 버리도록 코드 수정!
    finding = True
    where = 1
    try:
        while finding:
            where_is_review = '//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[{0}]'.format(where)

            please_review = browser.find_element(By.XPATH, value=where_is_review).text
            if please_review != '리뷰':
                where = where + 1

            else:
                link = browser.find_element(By.XPATH, value=where_is_review).get_attribute('href')
                finding = False
            
    except:
        print('{0}번 카페 {1} 마음에 안 들어요. 이 오류 수정하는 데만 1시간 걸렸거든요. 다음!'.format(num_iter, name))
        browser.switch_to.parent_frame()
        continue
    
    
    
    # 이 링크를 새로운 창으로 열겠습니다.
    # 그 과정을 볼게요.

    # 탭을 하나 더 열겠습니다. 그냥 구글으로 열었어요.
    browser.execute_script('window.open("https://google.com");')

    # 구글으로 열리기까지 시간을 좀 기다려줄게요.
    time.sleep(1)

    # 기존에 주피터가 조종하던 창은, 지도가 있고 프레임이 있는 탭이었는데
    # 이제 새로 연 탭을 주피터가 조종하도록 지시해줄게요.
    browser.switch_to.window(browser.window_handles[-1])

    # 새로연 탭을 조종할 수 있도록 했으니 이제 그 탭에서 저희가 가지고 있는 리뷰 링크를 열겠습니다.
    browser.get(link)
    time.sleep(2)

    ### 새로운 탭 열고 닫는 이유는,
    # 기존의 탭에서 링크를 접속할 수 있기는 한데 그러면 또다시 네이버맵에 들어가서 기다려야 한다는 점...
    # 그리고 뭐가 제대로 되고있는지 시각적으로 확인하기 위해서 새로운 창을 열었다 닫았다 하는 것도 있어요.
    # 이건 편하신 대로 하셔도 괜찮을 거 같아요! (by 승주)
    
    # 리뷰 수가 충분할까요?
    try:
        nado_button = browser.find_element(By.XPATH, value='//*[@id="app-root"]/div/div/div/div[7]/div/div[1]/div/div/div/div/div[3]/a')
        print('{0}번 카페 {1}에는 리뷰가 충분치 않습니다.(나도 버튼 없는 경우)'.format(num_iter, name))
        browser.close()
        browser.switch_to.window(browser.window_handles[0])
        continue

    except:
        pass
    
    try:
        visitor_button = browser.find_element(By.XPATH, value='//*[@id="app-root"]/div/div/div/div[7]/div[1]/div/a[1]')
    
    except:
        print('{0}번 카페 {1}에는 리뷰가 충분치 않습니다.(여러가지)'.format(num_iter, name))
        browser.close()
        browser.switch_to.window(browser.window_handles[0])
        continue
    
    # 이런점이 좋았어요 - 더보기버튼 누르기
    # 더보기버튼이 '더보기'일 때가 있고 '접기'일 때가 있습니다
    try:
        more = browser.find_element(by=By.XPATH, value='//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/a').text
    
    except:
        print('{0}번 카페 {1}에는 리뷰가 있는 것 같았지만 없습니다. 속았네요. 다음!'.format(num_iter, name))
        browser.close()
        browser.switch_to.window(browser.window_handles[0])
        continue

        
    # 더보기로 남아 있으면 한번 더 클릭!
    if more == '더보기':
        browser.find_element(by=By.XPATH, value='//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/a').click()
    time.sleep(1)
    
    # 더보기버튼은 다행히도 한 번 누른 뒤에도 XPATH가 바뀌지 않습니다. 얼마나 다행이게요? 
    # 한번더 check!

    more = browser.find_element(by=By.XPATH, value='//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/a').text
    if more == '더보기':
        browser.find_element(by=By.XPATH, value='//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/a').click()
    time.sleep(1)
    
    # 이 부분에서는 항목별 명칭 ('커피가 맛있어요' 등)과, 그 항목별 응답수 (25 등)를 크롤링합니다!
    # 단, 항목이 몇 종류가 기록되어 있는지에 따라 살짝 달라져야 해서 그 부분도 함께 조정합니다
    more = True
    num = 0
    key = []
    key_res = []
    while more:
        num = num+1
        val1 = '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/ul/li[{0}]/div[2]/span[1]'.format(num)
        val2 = '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/ul/li[{0}]/div[2]/span[2]'.format(num)
    
        try:
            keyword = browser.find_element(by=By.XPATH, value=val1).text
            key.append(keyword)
    
            digit_checker = len(browser.find_element(by=By.XPATH, value=val2).text)
            if digit_checker == 16:
                n_response = browser.find_element(by=By.XPATH, value=val2).text[-2:]
            else:
                n_response = browser.find_element(by=By.XPATH, value=val2).text[-1:]
    
            n_response = int(n_response)
            key_res.append(n_response)
        
        except:
            more = False
    
    # total (전체 응답 횟수)를 기록하기 위한 부분입니다!
    txt = browser.find_element(by=By.CSS_SELECTOR, value = 'div._Wmab').text
    cut = browser.find_element(by=By.CSS_SELECTOR, value = 'div._Wmab').text.index('회')
    tmp = ''
    for i in range(cut):
        tmp = tmp + txt[i]
        
    # 리뷰가 1,000개 이상이면 쉼표가 포함되는 불상사가 있어... 쉼표를 제거합니다.
    tmp = tmp.replace(',','')
    
    total = int(tmp)
    print('{0}번 카페 {1}에서는 총 {2}개의 항목과, {3}개의 응답이 발견되었습니다!'.format(num_iter, name, num-1, total))
    
    # 해당되는 카페에 대한게 맞는지 최종적으로 확인해 봅시다.
    search_name = browser.find_element(by=By.XPATH, value = '//*[@id="_title"]/span[1]').text
    
    # 하나의 카페에서 얻은 응답 결과를 딕셔너리 형태로 저장합니다!
    # 딕셔너리의 key는 항목 이름 + 'total',
    # 딕셔너리의 값은 항목별 응답 수 + 전체 응답 횟수 입니다!
    key_dict = dict(zip(key, key_res))
    key_dict['total_review'] = total
    key_dict['cafe'] = name
    key_dict['search_name'] = search_name
    key_dict['address'] = real_location
    key_dict['near_station'] = station
    key_dict['station_distance'] = station_distance
    key_dict['road_address'] = road_address
    
    # final 리스트에 딕셔너리를 append해서 한 주기를 마칩니다
    final.append(key_dict)
    browser.close()
    browser.switch_to.window(browser.window_handles[0])

# 모은 dictionary를 dataframe 형태로 바꾸기
final_df_col = ['cafe','search_name','address','road_address','near_station','station_distance','total_review',
            '"커피가 맛있어요"','"친절해요"','"음료가 맛있어요"', '"디저트가 맛있어요"',
            '"매장이 청결해요"', '"대화하기 좋아요"', '"집중하기 좋아요"','"인테리어가 멋져요"',
           '"좌석이 편해요"','"화장실이 깨끗해요"','"특별한 메뉴가 있어요"','"가성비가 좋아요"',
           '"뷰가 좋아요"', '"주차하기 편해요"', '"사진이 잘 나와요"']
final_df = pd.DataFrame(final, columns = final_df_col)

# 종료
# browser.quit()


end = time.time()
# 걸린 시간
print(f"{end - start:.5f} sec")

와! 12200개째 Crawling!
12200번 카페 모범시민카페에서는 총 13개의 항목과, 52개의 응답이 발견되었습니다!
12201번 카페 탐앤탐스디엠씨탐스커버리, 이 곳은 망한 것 같습니다.
12202번 카페 위드모리Withmorrie에서는 총 13개의 항목과, 40개의 응답이 발견되었습니다!
12203번 카페 지오반니, 이 곳은 망한 것 같습니다.
12204번 카페 ELCAFECOFFEEROASTERS에서는 총 14개의 항목과, 134개의 응답이 발견되었습니다!
12205번 카페 밀로커피에서는 총 14개의 항목과, 168개의 응답이 발견되었습니다!
12206번 카페 톡앤티에는 리뷰가 충분치 않습니다.(나도 버튼 없는 경우)
12207번 카페 카페한보리에는 리뷰가 충분치 않습니다.(여러가지)
12208번 카페 까페데미타스 페데미타스, 이 곳은 망한 것 같습니다.
12209번 카페 봄봄테이크아웃, 이 곳은 망한 것 같습니다.
12210번 카페 뎀셀브즈, 이 곳은 망한 것 같습니다.
12211번 카페 커피집, 이 곳은 망한 것 같습니다.
12212번 카페 비스트플래닛, 이 곳은 망한 것 같습니다.
12213번 카페 그릿시냇가까페에서는 총 15개의 항목과, 138개의 응답이 발견되었습니다!
12214번 카페 카페마노, 이 곳은 망한 것 같습니다.
12215번 카페 커피하우스, 이 곳은 망한 것 같습니다.
12216번 카페 우물가상사.커피넷,HOF이사, 이 곳은 망한 것 같습니다.
12217번 카페 커피공장, 이 곳은 망한 것 같습니다.
12218번 카페 힐링카페에코맘, 이 곳은 망한 것 같습니다.
12219번 카페 탐앤탐스 강남면허점에서는 총 14개의 항목과, 71개의 응답이 발견되었습니다!
151.50811 sec


In [ ]:
# 최종 얻게되는 dataframez
final_df

,cafe,search_name,address,road_address,near_station,station_distance,total_review,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""",...,"""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""","""특별한 메뉴가 있어요""","""가성비가 좋아요""","""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요"""
0,모범시민카페,모범시민카페,서초동,서울 서초구 강남대로47길 30,3신분당양재역,623m,52,28,17,8,...,5,1,2,2.0,1.0,3,7,NaN,NaN,1.0
1,위드모리Withmorrie,위드모리,신대방동,서울 동작구 보라매로 104,7신대방삼거리역,119m,40,22,9,15,...,8,2,3,2.0,NaN,2,9,1.0,NaN,1.0
2,ELCAFECOFFEEROASTERS,엘카페커피로스터스,성수동2가,서울 용산구 후암로 68 B1,14공항경의중앙서울역,469m,134,5,39,29,...,19,5,40,NaN,4.0,20,4,5.0,4.0,6.0
3,밀로커피,밀로커피 로스터스,동교동,서울 마포구 양화로18안길 36,2공항경의중앙홍대입구역,88m,168,8,43,38,...,21,9,16,5.0,9.0,21,2,2.0,NaN,4.0
4,그릿시냇가까페,그릿시냇가,신정동,None,None,None,138,81,22,22,...,24,9,24,7.0,7.0,7,19,3.0,9.0,9.0
5,탐앤탐스 강남면허점,탐앤탐스커피강남면허점,대치동,서울 강남구 테헤란로114길 16 동남빌딩 1층,2삼성역,351m,71,35,24,22,...,17,1,3,18.0,8.0,3,8,1.0,1.0,NaN


In [ ]:
### 경로 지정 조심해주세요!!
# 다 되면 dataframe을 csv로 저장하기
# for문을 어디에 대해서 시행했는지 0-10처럼 옆에 적어주시면 데이터를 합칠 때 훨씬 좋을 것 같습니다!


final_df.to_csv('C:\\Users\\김지희\\modeling_project\\dataset\\naver12100-12200.csv', index=False)

In [ ]:
# 저장하고 난후 데이터를 다시 불러와서 잘 크롤링되었는지 확인해봅시다!
df_made = pd.read_csv('C:\\Users\\김지희\\modeling_project\\dataset\\naver12100-12200.csv')   # 여기 수정!
df_made

,cafe,search_name,address,near_station,station_distance,total_review,"""커피가 맛있어요""","""친절해요""","""음료가 맛있어요""","""디저트가 맛있어요""",...,"""대화하기 좋아요""","""집중하기 좋아요""","""인테리어가 멋져요""","""좌석이 편해요""","""화장실이 깨끗해요""","""특별한 메뉴가 있어요""","""가성비가 좋아요""","""뷰가 좋아요""","""주차하기 편해요""","""사진이 잘 나와요"""
0,카페 그린버디,카페그린버디,도봉동,17도봉산역,607m,101,60.0,22.0,21.0,6.0,...,8.0,1.0,5.0,1.0,1.0,1.0,18.0,13.0,1.0,1.0
1,엠지엠지,엠지엠지,성내동,5둔촌동역,592m,22,13.0,9.0,3.0,9.0,...,1.0,NaN,1.0,NaN,NaN,3.0,2.0,NaN,NaN,NaN
2,다솜케이크디저트커피,베이커리 다솜,미아동,NaN,NaN,1083,8.0,3.0,85.0,NaN,...,49.0,17.0,8.0,20.0,10.0,7.0,6.0,16.0,10.0,41.0
3,레어마카롱,레어마카롱 대치점,대치동,수인분당한티역,775m,42,14.0,12.0,7.0,10.0,...,1.0,1.0,9.0,1.0,NaN,8.0,2.0,2.0,NaN,4.0
4,마일스톤커피로스터스,마일스톤 커피 한남점,신사동,6한강진역,310m,546,3.0,9.0,88.0,4.0,...,28.0,1.0,81.0,6.0,10.0,48.0,9.0,10.0,3.0,46.0
5,월디커피로스터스,월디커피로스터스,둔촌동,5둔촌동역,721m,49,36.0,16.0,14.0,6.0,...,5.0,3.0,8.0,1.0,3.0,5.0,6.0,1.0,NaN,3.0
6,카페울림,카페울림,신월동,NaN,NaN,16,12.0,6.0,6.0,3.0,...,2.0,NaN,1.0,1.0,4.0,2.0,3.0,NaN,NaN,NaN
7,라이크어시네마,라이크어시네마,등촌동,9등촌역,685m,50,3.0,15.0,5.0,9.0,...,NaN,NaN,12.0,NaN,NaN,20.0,18.0,NaN,1.0,5.0
8,보니또커피샤로수점,보니또커피 샤로수길점,봉천동,2서울대입구역,378m,155,68.0,31.0,28.0,31.0,...,3.0,NaN,9.0,1.0,NaN,17.0,47.0,NaN,NaN,1.0
9,함께그린카페금나래문화체육센터점,함께그린카페 금나래문화체육센터점,독산동,1금천구청역,438m,118,83.0,24.0,13.0,12.0,...,8.0,4.0,10.0,3.0,4.0,2.0,15.0,1.0,NaN,2.0
